## Fama French 3 Factor Model Data
Past 7 years

In [ ]:
import pandas as pd
import requests
import zipfile
import io
from io import StringIO

url = "https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_CSV.zip"
r = requests.get(url)

z = zipfile.ZipFile(io.BytesIO(r.content))
csv_data = z.open("F-F_Research_Data_Factors_daily.csv").read().decode("utf-8")

df = pd.read_csv(StringIO(csv_data), skiprows=4)

df.rename(columns={df.columns[0]: 'Date'}, inplace=True)

df = df[df['Date'].astype(str).str.len() == 8]

df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')
df.set_index('Date', inplace=True)
df.sort_index(inplace=True)

df = df / 100.0

cutoff = pd.Timestamp.today().normalize() - pd.DateOffset(years=7)
df_last7 = df.loc[cutoff:]

df_last7

,Mkt-RF,SMB,HML,RF
Date,,,,
2019-01-22,-0.0152,-0.0039,0.0030,0.0001
2019-01-23,0.0016,-0.0038,-0.0015,0.0001
2019-01-24,0.0023,0.0044,0.0000,0.0001
2019-01-25,0.0090,0.0046,-0.0034,0.0001
2019-01-28,-0.0080,-0.0016,0.0063,0.0001
...,...,...,...,...
2025-11-21,0.0103,0.0166,0.0074,0.0002
2025-11-24,0.0161,0.0030,-0.0096,0.0002
2025-11-25,0.0104,0.0165,0.0004,0.0002


## Fama French 5 Factor Model Data
Past 7 years

In [16]:
import pandas as pd
import requests
import zipfile
import io
from io import StringIO

url = "https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_5_Factors_2x3_daily_CSV.zip"
r = requests.get(url)

z = zipfile.ZipFile(io.BytesIO(r.content))
csv_data = z.open("F-F_Research_Data_5_Factors_2x3_daily.csv").read().decode("utf-8")

df = pd.read_csv(StringIO(csv_data), skiprows=3)

df.rename(columns={df.columns[0]: 'Date'}, inplace=True)

df = df[df['Date'].astype(str).str.len() == 8]

df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')
df.set_index('Date', inplace=True)
df.sort_index(inplace=True)

df = df / 100.0

cutoff = pd.Timestamp.today().normalize() - pd.DateOffset(years=7)
df_last7 = df.loc[cutoff:]

df_last7

,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,
2019-01-22,-0.0152,-0.0035,0.0030,0.0026,0.0036,0.0001
2019-01-23,0.0016,-0.0041,-0.0015,0.0035,0.0027,0.0001
2019-01-24,0.0023,0.0048,0.0000,-0.0015,-0.0023,0.0001
2019-01-25,0.0090,0.0041,-0.0034,-0.0036,-0.0057,0.0001
2019-01-28,-0.0080,-0.0007,0.0063,0.0029,0.0017,0.0001
...,...,...,...,...,...,...
2025-11-21,0.0103,0.0185,0.0074,0.0009,0.0095,0.0002
2025-11-24,0.0161,0.0005,-0.0096,-0.0108,-0.0157,0.0002
2025-11-25,0.0104,0.0158,0.0004,-0.0009,0.0036,0.0002


## Encapsulated function to pull Fama French data

In [22]:
import pandas as pd
import requests
import zipfile
import io
from io import StringIO


def fetch_fama_french_factors(
    n_factors: int = 3,
    start_date: str | None = None,
    end_date: str | None = None,
    as_decimal: bool = True,
) -> pd.DataFrame:

    if n_factors == 3:
        url = (
            "https://mba.tuck.dartmouth.edu/pages/faculty/"
            "ken.french/ftp/F-F_Research_Data_Factors_daily_CSV.zip"
        )
        csv_name = "F-F_Research_Data_Factors_daily.csv"
        skiprows = 4

    elif n_factors == 5:
        url = (
            "https://mba.tuck.dartmouth.edu/pages/faculty/"
            "ken.french/ftp/F-F_Research_Data_5_Factors_2x3_daily_CSV.zip"
        )
        csv_name = "F-F_Research_Data_5_Factors_2x3_daily.csv"
        skiprows = 3

    else:
        raise ValueError("n_factors must be 3 or 5")

    r = requests.get(url)
    r.raise_for_status()

    z = zipfile.ZipFile(io.BytesIO(r.content))
    csv_data = z.open(csv_name).read().decode("utf-8")

    df = pd.read_csv(StringIO(csv_data), skiprows=skiprows)

    df.rename(columns={df.columns[0]: "Date"}, inplace=True)

    df = df[df["Date"].astype(str).str.len() == 8]

    df["Date"] = pd.to_datetime(df["Date"], format="%Y%m%d")
    df.set_index("Date", inplace=True)
    df.sort_index(inplace=True)

    if as_decimal:
        df = df / 100.0

    if start_date is not None:
        df = df.loc[pd.to_datetime(start_date):]

    if end_date is not None:
        df = df.loc[:pd.to_datetime(end_date)]

    return df

In [23]:
ff3 = fetch_fama_french_factors(
    n_factors=3,
    start_date="2019-01-22"
)

ff3

,Mkt-RF,SMB,HML,RF
Date,,,,
2019-01-22,-0.0152,-0.0039,0.0030,0.0001
2019-01-23,0.0016,-0.0038,-0.0015,0.0001
2019-01-24,0.0023,0.0044,0.0000,0.0001
2019-01-25,0.0090,0.0046,-0.0034,0.0001
2019-01-28,-0.0080,-0.0016,0.0063,0.0001
...,...,...,...,...
2025-11-21,0.0103,0.0166,0.0074,0.0002
2025-11-24,0.0161,0.0030,-0.0096,0.0002
2025-11-25,0.0104,0.0165,0.0004,0.0002


In [24]:
ff5 = fetch_fama_french_factors(
    n_factors=5,
    start_date="2019-01-22"
)

ff5

,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,
2019-01-22,-0.0152,-0.0035,0.0030,0.0026,0.0036,0.0001
2019-01-23,0.0016,-0.0041,-0.0015,0.0035,0.0027,0.0001
2019-01-24,0.0023,0.0048,0.0000,-0.0015,-0.0023,0.0001
2019-01-25,0.0090,0.0041,-0.0034,-0.0036,-0.0057,0.0001
2019-01-28,-0.0080,-0.0007,0.0063,0.0029,0.0017,0.0001
...,...,...,...,...,...,...
2025-11-21,0.0103,0.0185,0.0074,0.0009,0.0095,0.0002
2025-11-24,0.0161,0.0005,-0.0096,-0.0108,-0.0157,0.0002
2025-11-25,0.0104,0.0158,0.0004,-0.0009,0.0036,0.0002
